<a href="https://colab.research.google.com/github/High-ren/Classification/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
x=7
y=9

In [2]:
x+y

16